In [1]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTy1ZbSq5OAVbEqScDw_Uf_KfLKydg0uUWBaBEPK_DXaBPCHmwv4ru9MzZHHnVHVmRVvLvOlndJ2ZNZ/pubhtml?gid=159569114&amp;single=true&amp;widget=true&amp;headers=false&amp;gid=0&amp;range=A:F%22'
res = requests.get(url)

In [3]:
#　抽出したHTMLの内容を確認する。
res.text[:500]

'<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta name="viewport" content="target-densitydpi=device-dpi,user-scalable=1,minimum-scale=1,maximum-scale=2.5,initial-scale=1,width=device-width"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><meta name="referrer" content="origin"><title>カードラッシュ\u3000買取表 - Google ドライブ</title><link rel="shortcut icon" href="//ssl.gstatic.com/docs/spreadsheets/favicon3.ico"><link href=\'/static/spreadsheets2/client/css/316073865-waffle_k_ltr.cs'

In [4]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.text,'html.parser')

In [67]:
# 最新弾の情報を抽出する。
poke_new = soup.find('div',id ='1490875147')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_new.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,349):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(0)

#価格が欠損値になっている行を削除する。
df_new = df_new.dropna(subset = ['買取'])
21
#買取データの日付を取得する。
import datetime 
today = pd.to_datetime('2021-01-25')
today = today.strftime('%Y年%m月%d日')

df_new.columns = [['カード名', '仕様', 'R', '型番', 'タイプ',today]]

In [68]:
#### 通常の情報を抽出する。
poke_norm = soup.find('div',id ='159569114')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_norm.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,3644):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)

# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_norm.columns = df_norm.iloc[0]
df_norm = df_norm.drop(0)

#価格が欠損値になっている行を削除する。
df_norm = df_norm.dropna(subset = ['買取価格'])
21
#買取データの日付を取得する。
import datetime 
today = pd.to_datetime('2021-01-25')
today = today.strftime('%Y年%m月%d日')

df_norm.columns = [['カード名', '仕様', 'R', '型番', 'タイプ',today]]

In [71]:
df_all = pd.concat([df_norm,df_new],ignore_index = True)
df_all

カード名    仕様   R       型番 タイプ 2021年01月25日
0       フシギバナ＆ツタージャGX    SA  SR  066/064   草       1,200
1     モクロー＆アローラナッシーGX    SA  SR  056/054   草       3,500
2     フェローチェ＆マッシブーンGX    SA  SR  056/054   草       1,100
3        セレビィ＆フシギバナGX    SA  SR  097/095   草       1,300
4       リザードン＆テールナーGX    SA  SR  068/064   炎       3,800
...               ...   ...  ..      ...  ..         ...
3639            キングドラ  None   R  019/070   水          10
3640             オクタン  None   R  023/070   水         100
3641             イオルブ  None   R  038/070   超          10
3642           エンニュート  None   U  014/070   炎          10
3643             カビゴン  None   P  156/S-P   無        1000

[3644 rows x 6 columns]

In [72]:
df_all.to_csv('all_card_data.csv',index = None)